In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('DATABEFOREPERCENTILE.csv')

In [3]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
mis = data.isnull().sum()
mis

Country Name                                                                0
Time                                                                        0
Exports of goods and services (% of GDP)                                    0
Foreign direct investment, net inflows (% of GDP)                           0
GDP (current US$)                                                           0
GDP growth (annual %)                                                       0
GDP per capita (current US$)                                                0
GDP per capita growth (annual %)                                            0
Inflation, GDP deflator (annual %)                                          0
Trade (% of GDP)                                                            0
Imports of goods and services (% of GDP)                                    0
Gross capital formation (annual % growth)                                   0
Unemployment, total (% of total labor force) (modeled ILO estima

In [6]:
def sanitize_country_name(name):
    return (
        name.lower()
        .replace(' ', '')
        .replace('-', '')
        .replace('(', '')
        .replace(')', '')
        .replace(',', '')
        .replace('.', '')
        .replace("'", '')
    )

country_names = data['Country Name'].unique()
country_var_names = [f"df_{sanitize_country_name(country)}" for country in country_names]

for country, var_name in zip(country_names, country_var_names):
    sanitized_country = country.replace("'", "\\'")
    exec(f"{var_name} = data[data['Country Name'] == '{sanitized_country}'].reset_index(drop=True)")

print("DataFrames created for each country:")
for name in country_var_names:
    print(name)

DataFrames created for each country:
df_algeria
df_angola
df_benin
df_botswana
df_burkinafaso
df_burundi
df_cameroon
df_congodemrep
df_egyptarabrep
df_ghana
df_guinea
df_kenya
df_madagascar
df_mauritania
df_mauritius
df_morocco
df_namibia
df_rwanda
df_senegal
df_southafrica
df_sudan
df_tanzania
df_togo
df_tunisia
df_uganda


In [8]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

best_params = {
    'smoothing_level': 0.4,
    'smoothing_slope': 0.6,
    'damping_slope': 0.8
}

def apply_holt_linear_trend(df, indicator):
    df = df.copy()
    df['Time'] = pd.to_datetime(df['Time'], format='%Y')
    df = df.set_index('Time')
    y = df[indicator]
    
    model = ExponentialSmoothing(y, trend='add', damped_trend=True, seasonal=None)
    fit = model.fit(
        smoothing_level=best_params['smoothing_level'],
        smoothing_slope=best_params['smoothing_slope'],
        damping_slope=best_params['damping_slope']
    )
    
    future_years = pd.date_range(start='2023', end='2027', freq='YS')
    future_predictions = fit.forecast(steps=len(future_years))
    
    future_df = pd.DataFrame(future_predictions, index=future_years, columns=[indicator])
    future_df['Time'] = future_df.index.year + 0.0
    
    return future_df

def forecast_all_indicators(df):
    future_years = pd.date_range(start='2023', end='2027', freq='YS')
    df_forecast = pd.DataFrame(index=future_years)
    
    for indicator in df.columns:
        if indicator not in ['Time', 'Country Name']:
            future_df = apply_holt_linear_trend(df, indicator)
            df_forecast[indicator] = future_df[indicator]
    
    df_forecast['Country Name'] = df['Country Name'].iloc[0]
    df_forecast['Time'] = df_forecast.index.year + 0.0
    
    return df_forecast

df_algeria_forecast = forecast_all_indicators(df_algeria)
df_algeria_forecast


/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 27.502616   
2024-01-01                                 29.572969   
2025-01-01                                 31.229251   
2026-01-01                                 32.554277   
2027-01-01                                 33.614297   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           0.260709   
2024-01-01                                           0.152651   
2025-01-01                                           0.066204   
2026-01-01                                          -0.002953   
2027-01-01                                          -0.058279   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       2.042146e+11               2.108947   
2024-01-01       2.109502e+11               2.750763   
2025-01-01       2.163386e+11               3.264216   
2026-01-01       2.206494e+11               3.674978   
2027-01-01       2.240980e+11               4.003588   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   4514.245550                          0.498525   
2024-01-01                   4627.231682                          1.172655   
2025-01-01                   4717.620588                          1.711960   
2026-01-01                   4789.931712                          2.143403   
2027-01-01                   4847.780612                          2.488558   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                           18.759863         47.898213   
2024-01-01                           22.533933         48.688004   
2025-01-01                           25.553188         49.319837   
2026-01-01                           27.968593         49.825303   
2027-01-01                           29.900916         50.229676   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 20.395597   
2024-01-01                                 19.115035   
2025-01-01                                 18.090586   
2026-01-01                                 17.271026   
2027-01-01                                 16.615378   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  -0.065346  ...   
2024-01-01                                   1.317297  ...   
2025-01-01                                   2.423412  ...   
2026-01-01                                   3.308303  ...   
2027-01-01                                   4.016217  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      106.072610   
2024-01-01                                      105.811183   
2025-01-01                                      105.602041   
2026-01-01                                      105.434728   
2027-01-01                                      105.300877   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              55.366260       4.513329e+07   
2024-01-01                              55.380674       4.555027e+07   
2025-01-01                              55.392204       4.588385e+07   
2026-01-01                              55.401429       4.615071e+07   
2027-01-01                              55.408809       4.636420e+07   

            Urban population growth (annual %)  \
2023-01-01                            2.269209   
2024-01-01                            2.210757   
2025-01-01                            2.163995   
2026-01-01                            2.126586   
2027-01-01                            2.096659   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.352686   
2024-01-01                                         -0.463651   
2025-01-01                                         -0.552422

In [9]:
df_angola_forecast = forecast_all_indicators(df_angola)
df_angola_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 47.649844   
2024-01-01                                 49.462709   
2025-01-01                                 50.913002   
2026-01-01                                 52.073236   
2027-01-01                                 53.001423   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                          -6.843840   
2024-01-01                                          -6.848700   
2025-01-01                                          -6.852589   
2026-01-01                                          -6.855699   
2027-01-01                                          -6.858187   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       8.324356e+10               1.303129   
2024-01-01       8.982723e+10               2.254224   
2025-01-01       9.509416e+10               3.015099   
2026-01-01       9.930771e+10               3.623800   
2027-01-01       1.026785e+11               4.110761   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   2296.216290                         -1.756434   
2024-01-01                   2456.610137                         -0.782000   
2025-01-01                   2584.925214                         -0.002453   
2026-01-01                   2687.577276                          0.621185   
2027-01-01                   2769.698926                          1.120095   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                           22.696687         73.586577   
2024-01-01                           22.104079         76.111937   
2025-01-01                           21.629992         78.132225   
2026-01-01                           21.250722         79.748455   
2027-01-01                           20.947306         81.041440   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 25.936733   
2024-01-01                                 26.649227   
2025-01-01                                 27.219223   
2026-01-01                                 27.675219   
2027-01-01                                 28.040016   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   6.838152  ...   
2024-01-01                                   8.510550  ...   
2025-01-01                                   9.848468  ...   
2026-01-01                                  10.918802  ...   
2027-01-01                                  11.775069  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       57.632827   
2024-01-01                                       61.338531   
2025-01-01                                       64.303094   
2026-01-01                                       66.674745   
2027-01-01                                       68.572065   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               5.580197       3.587122e+07   
2024-01-01                               4.958041       3.648365e+07   
2025-01-01                               4.460317       3.697359e+07   
2026-01-01                               4.062137       3.736555e+07   
2027-01-01                               3.743594       3.767911e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.972156   
2024-01-01                            3.899201   
2025-01-01                            3.840838   
2026-01-01                            3.794147   
2027-01-01                            3.756794   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.398150   
2024-01-01                                         -0.493821   
2025-01-01                                         -0.570358

In [10]:
df_benin_forecast = forecast_all_indicators(df_benin)
df_benin_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 19.987907   
2024-01-01                                 19.023697   
2025-01-01                                 18.252329   
2026-01-01                                 17.635235   
2027-01-01                                 17.141559   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.601875   
2024-01-01                                           1.664045   
2025-01-01                                           1.713780   
2026-01-01                                           1.753569   
2027-01-01                                           1.785400   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.810611e+10               6.500021   
2024-01-01       1.871281e+10               6.626895   
2025-01-01       1.919817e+10               6.728394   
2026-01-01       1.958646e+10               6.809593   
2027-01-01       1.989709e+10               6.874553   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   1352.704713                          3.659028   
2024-01-01                   1381.208672                          3.819016   
2025-01-01                   1404.011840                          3.947006   
2026-01-01                   1422.254373                          4.049398   
2027-01-01                   1436.848401                          4.131312   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            3.624674         46.811003   
2024-01-01                            4.260735         45.153227   
2025-01-01                            4.769584         43.827006   
2026-01-01                            5.176663         42.766029   
2027-01-01                            5.502327         41.917247   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 26.823097   
2024-01-01                                 26.129530   
2025-01-01                                 25.574677   
2026-01-01                                 25.130794   
2027-01-01                                 24.775688   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  12.089418  ...   
2024-01-01                                  12.485477  ...   
2025-01-01                                  12.802324  ...   
2026-01-01                                  13.055801  ...   
2027-01-01                                  13.258584  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      105.910211   
2024-01-01                                      110.796946   
2025-01-01                                      114.706334   
2026-01-01                                      117.833845   
2027-01-01                                      120.335854   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               9.812420       1.344326e+07   
2024-01-01                               9.638332       1.364460e+07   
2025-01-01                               9.499061       1.380567e+07   
2026-01-01                               9.387645       1.393452e+07   
2027-01-01                               9.298512       1.403760e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.844157   
2024-01-01                            3.806300   
2025-01-01                            3.776014   
2026-01-01                            3.751786   
2027-01-01                            3.732403   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.389433   
2024-01-01                                         -0.527630   
2025-01-01                                         -0.638188

In [11]:
df_botswana_forecast = forecast_all_indicators(df_botswana)
df_botswana_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 38.578346   
2024-01-01                                 39.229890   
2025-01-01                                 39.751126   
2026-01-01                                 40.168114   
2027-01-01                                 40.501704   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                          -0.374954   
2024-01-01                                          -0.460884   
2025-01-01                                          -0.529628   
2026-01-01                                          -0.584623   
2027-01-01                                          -0.628619   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.955130e+10               5.554039   
2024-01-01       2.024033e+10               6.540891   
2025-01-01       2.079155e+10               7.330373   
2026-01-01       2.123253e+10               7.961958   
2027-01-01       2.158531e+10               8.467227   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   7394.134395                          3.896334   
2024-01-01                   7593.324003                          4.938308   
2025-01-01                   7752.675689                          5.771888   
2026-01-01                   7880.157038                          6.438751   
2027-01-01                   7982.142118                          6.972242   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                           12.084958         82.480229   
2024-01-01                           14.386552         83.192758   
2025-01-01                           16.227828         83.762780   
2026-01-01                           17.700848         84.218799   
2027-01-01                           18.879265         84.583613   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 43.901882   
2024-01-01                                 43.962867   
2025-01-01                                 44.011655   
2026-01-01                                 44.050685   
2027-01-01                                 44.081909   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  -4.580625  ...   
2024-01-01                                  -7.942561  ...   
2025-01-01                                 -10.632109  ...   
2026-01-01                                 -12.783748  ...   
2027-01-01                                 -14.505059  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      164.846784   
2024-01-01                                      168.860535   
2025-01-01                                      172.071536   
2026-01-01                                      174.640336   
2027-01-01                                      176.695377   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              19.963408       2.646159e+06   
2024-01-01                              19.124186       2.671309e+06   
2025-01-01                              18.452808       2.691429e+06   
2026-01-01                              17.915705       2.707525e+06   
2027-01-01                              17.486023       2.720401e+06   

            Urban population growth (annual %)  \
2023-01-01                            2.525506   
2024-01-01                            2.439300   
2025-01-01                            2.370336   
2026-01-01                            2.315164   
2027-01-01                            2.271027   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.387427   
2024-01-01                                         -0.506941   
2025-01-01                                         -0.602552

In [12]:
df_burkinafaso_forecast = forecast_all_indicators(df_burkinafaso)
df_burkinafaso_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 33.001023   
2024-01-01                                 33.579938   
2025-01-01                                 34.043069   
2026-01-01                                 34.413574   
2027-01-01                                 34.709978   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.470913   
2024-01-01                                           1.859433   
2025-01-01                                           2.170249   
2026-01-01                                           2.418902   
2027-01-01                                           2.617824   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.991888e+10               3.197582   
2024-01-01       2.050146e+10               2.699768   
2025-01-01       2.096752e+10               2.301516   
2026-01-01       2.134038e+10               1.982915   
2027-01-01       2.163866e+10               1.728034   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    877.259923                          0.598207   
2024-01-01                    892.563109                          0.146392   
2025-01-01                    904.805657                         -0.215060   
2026-01-01                    914.599695                         -0.504222   
2027-01-01                    922.434926                         -0.735552   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            4.641016         63.919401   
2024-01-01                            5.188814         64.734301   
2025-01-01                            5.627053         65.386221   
2026-01-01                            5.977644         65.907757   
2027-01-01                            6.258117         66.324985   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 30.918377   
2024-01-01                                 31.154363   
2025-01-01                                 31.343151   
2026-01-01                                 31.494182   
2027-01-01                                 31.615007   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   3.141720  ...   
2024-01-01                                   3.100266  ...   
2025-01-01                                   3.067102  ...   
2026-01-01                                   3.040571  ...   
2027-01-01                                   3.019347  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       78.157648   
2024-01-01                                       71.240150   
2025-01-01                                       65.706151   
2026-01-01                                       61.278952   
2027-01-01                                       57.737193   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               9.963138       2.282006e+07   
2024-01-01                              10.502998       2.314361e+07   
2025-01-01                              10.934886       2.340245e+07   
2026-01-01                              11.280397       2.360952e+07   
2027-01-01                              11.556805       2.377518e+07   

            Urban population growth (annual %)  \
2023-01-01                            4.562341   
2024-01-01                            4.513555   
2025-01-01                            4.474526   
2026-01-01                            4.443303   
2027-01-01                            4.418325   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.173759   
2024-01-01                                         -0.296618   
2025-01-01                                         -0.394905

In [13]:
df_burundi_forecast = forecast_all_indicators(df_burundi)
df_burundi_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                  4.637480   
2024-01-01                                  4.617659   
2025-01-01                                  4.601802   
2026-01-01                                  4.589117   
2027-01-01                                  4.578969   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           0.264028   
2024-01-01                                           0.349357   
2025-01-01                                           0.417621   
2026-01-01                                           0.472232   
2027-01-01                                           0.515920   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       3.065824e+09               2.554448   
2024-01-01       3.170519e+09               2.836115   
2025-01-01       3.254275e+09               3.061448   
2026-01-01       3.321280e+09               3.241715   
2027-01-01       3.374884e+09               3.385929   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    235.299439                         -0.320966   
2024-01-01                    239.866370                         -0.042097   
2025-01-01                    243.519915                          0.180998   
2026-01-01                    246.442752                          0.359474   
2027-01-01                    248.781021                          0.502255   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                           13.802109         29.182881   
2024-01-01                           16.430535         29.949449   
2025-01-01                           18.533276         30.562704   
2026-01-01                           20.215469         31.053308   
2027-01-01                           21.561223         31.445791   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 24.545401   
2024-01-01                                 25.331790   
2025-01-01                                 25.960902   
2026-01-01                                 26.464191   
2027-01-01                                 26.866822   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   3.642170  ...   
2024-01-01                                   2.346138  ...   
2025-01-01                                   1.309311  ...   
2026-01-01                                   0.479851  ...   
2027-01-01                                  -0.183718  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       60.008859   
2024-01-01                                       60.339211   
2025-01-01                                       60.603492   
2026-01-01                                       60.814918   
2027-01-01                                       60.984058   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               6.642270       1.296975e+07   
2024-01-01                               6.886254       1.316850e+07   
2025-01-01                               7.081441       1.332749e+07   
2026-01-01                               7.237590       1.345468e+07   
2027-01-01                               7.362510       1.355644e+07   

            Urban population growth (annual %)  \
2023-01-01                            5.368088   
2024-01-01                            5.361458   
2025-01-01                            5.356154   
2026-01-01                            5.351911   
2027-01-01                            5.348517   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.207738   
2024-01-01                                         -0.323668   
2025-01-01                                         -0.416413

In [14]:
df_cameroon_forecast = forecast_all_indicators(df_cameroon)
df_cameroon_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 17.436208   
2024-01-01                                 17.599007   
2025-01-01                                 17.729247   
2026-01-01                                 17.833438   
2027-01-01                                 17.916791   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           2.069911   
2024-01-01                                           2.066931   
2025-01-01                                           2.064546   
2026-01-01                                           2.062639   
2027-01-01                                           2.061113   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       4.516130e+10               2.604574   
2024-01-01       4.602294e+10               2.692505   
2025-01-01       4.671225e+10               2.762849   
2026-01-01       4.726370e+10               2.819125   
2027-01-01       4.770486e+10               2.864146   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   1610.612971                          0.008878   
2024-01-01                   1619.830125                          0.129756   
2025-01-01                   1627.203849                          0.226458   
2026-01-01                   1633.102828                          0.303819   
2027-01-01                   1637.822011                          0.365709   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            4.273787         38.070201   
2024-01-01                            4.988149         38.338763   
2025-01-01                            5.559638         38.553613   
2026-01-01                            6.016830         38.725493   
2027-01-01                            6.382583         38.862996   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 20.633993   
2024-01-01                                 20.739756   
2025-01-01                                 20.824366   
2026-01-01                                 20.892054   
2027-01-01                                 20.946205   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  -1.106803  ...   
2024-01-01                                  -2.214725  ...   
2025-01-01                                  -3.101063  ...   
2026-01-01                                  -3.810134  ...   
2027-01-01                                  -4.377390  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       82.210856   
2024-01-01                                       82.319694   
2025-01-01                                       82.406766   
2026-01-01                                       82.476422   
2027-01-01                                       82.532148   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               9.613335       2.809687e+07   
2024-01-01                               9.340631       2.849791e+07   
2025-01-01                               9.122468       2.881874e+07   
2026-01-01                               8.947937       2.907540e+07   
2027-01-01                               8.808313       2.928073e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.561369   
2024-01-01                            3.517427   
2025-01-01                            3.482273   
2026-01-01                            3.454149   
2027-01-01                            3.431651   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.414566   
2024-01-01                                         -0.554479   
2025-01-01                                         -0.666409

In [15]:
df_egyptarabrep_forecast = forecast_all_indicators(df_egyptarabrep)
df_egyptarabrep_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 13.311946   
2024-01-01                                 13.139239   
2025-01-01                                 13.001073   
2026-01-01                                 12.890540   
2027-01-01                                 12.802114   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.735805   
2024-01-01                                           1.606948   
2025-01-01                                           1.503862   
2026-01-01                                           1.421394   
2027-01-01                                           1.355419   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       4.708646e+11               5.162489   
2024-01-01       5.007720e+11               5.342089   
2025-01-01       5.246980e+11               5.485769   
2026-01-01       5.438387e+11               5.600713   
2027-01-01       5.591513e+11               5.692668   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   4233.679153                          3.563753   
2024-01-01                   4478.698685                          3.789580   
2025-01-01                   4674.714310                          3.970242   
2026-01-01                   4831.526810                          4.114771   
2027-01-01                   4956.976810                          4.230395   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            6.124388         33.463462   
2024-01-01                            4.779772         32.667266   
2025-01-01                            3.704080         32.030308   
2026-01-01                            2.843526         31.520743   
2027-01-01                            2.155083         31.113090   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 20.151516   
2024-01-01                                 19.528027   
2025-01-01                                 19.029235   
2026-01-01                                 18.630202   
2027-01-01                                 18.310976   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   3.424860  ...   
2024-01-01                                   4.630998  ...   
2025-01-01                                   5.595908  ...   
2026-01-01                                   6.367837  ...   
2027-01-01                                   6.985379  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       91.519014   
2024-01-01                                       91.908686   
2025-01-01                                       92.220424   
2026-01-01                                       92.469814   
2027-01-01                                       92.669326   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              39.460347       1.115647e+08   
2024-01-01                              39.683408       1.125674e+08   
2025-01-01                              39.861856       1.133696e+08   
2026-01-01                              40.004615       1.140114e+08   
2027-01-01                              40.118822       1.145248e+08   

            Urban population growth (annual %)  \
2023-01-01                            1.796167   
2024-01-01                            1.787570   
2025-01-01                            1.780693   
2026-01-01                            1.775191   
2027-01-01                            1.770789   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                          0.023295   
2024-01-01                                         -0.002927   
2025-01-01                                         -0.023905

In [16]:
df_ghana_forecast = forecast_all_indicators(df_ghana)
df_ghana_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 32.866638   
2024-01-01                                 32.799213   
2025-01-01                                 32.745274   
2026-01-01                                 32.702122   
2027-01-01                                 32.667601   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           2.066341   
2024-01-01                                           1.629085   
2025-01-01                                           1.279280   
2026-01-01                                           0.999437   
2027-01-01                                           0.775562   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       7.839163e+10               3.639938   
2024-01-01       7.989599e+10               3.470649   
2025-01-01       8.109948e+10               3.335218   
2026-01-01       8.206227e+10               3.226873   
2027-01-01       8.283250e+10               3.140198   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   2334.653125                          1.665940   
2024-01-01                   2356.262479                          1.531701   
2025-01-01                   2373.549962                          1.424310   
2026-01-01                   2387.379948                          1.338397   
2027-01-01                   2398.443938                          1.269667   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                           20.320297         67.100081   
2024-01-01                           23.424477         66.843482   
2025-01-01                           25.907820         66.638203   
2026-01-01                           27.894495         66.473980   
2027-01-01                           29.483834         66.342601   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 34.233443   
2024-01-01                                 34.044269   
2025-01-01                                 33.892929   
2026-01-01                                 33.771858   
2027-01-01                                 33.675000   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   1.334316  ...   
2024-01-01                                   1.354075  ...   
2025-01-01                                   1.369882  ...   
2026-01-01                                   1.382528  ...   
2027-01-01                                   1.392645  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      120.649564   
2024-01-01                                      118.618252   
2025-01-01                                      116.993203   
2026-01-01                                      115.693164   
2027-01-01                                      114.653132   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              20.323957       3.365491e+07   
2024-01-01                              20.856926       3.402081e+07   
2025-01-01                              21.283302       3.431353e+07   
2026-01-01                              21.624402       3.454771e+07   
2027-01-01                              21.897283       3.473505e+07   

            Urban population growth (annual %)  \
2023-01-01                            2.997321   
2024-01-01                            2.953535   
2025-01-01                            2.918506   
2026-01-01                            2.890483   
2027-01-01                            2.868064   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.360750   
2024-01-01                                         -0.492857   
2025-01-01                                         -0.598542

In [17]:
df_guinea_forecast = forecast_all_indicators(df_guinea)
df_guinea_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 45.749929   
2024-01-01                                 46.374431   
2025-01-01                                 46.874033   
2026-01-01                                 47.273715   
2027-01-01                                 47.593460   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.055112   
2024-01-01                                           0.909322   
2025-01-01                                           0.792689   
2026-01-01                                           0.699383   
2027-01-01                                           0.624739   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.989409e+10               3.672729   
2024-01-01       2.131451e+10               3.188918   
2025-01-01       2.245085e+10               2.801868   
2026-01-01       2.335992e+10               2.492229   
2027-01-01       2.408717e+10               2.244517   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   1437.124137                          1.211425   
2024-01-01                   1526.777089                          0.767731   
2025-01-01                   1598.499450                          0.412776   
2026-01-01                   1655.877339                          0.128812   
2027-01-01                   1701.779650                         -0.098360   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                           11.006622        109.683346   
2024-01-01                           11.608374        111.857589   
2025-01-01                           12.089776        113.596983   
2026-01-01                           12.474897        114.988498   
2027-01-01                           12.782994        116.101710   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 63.933417   
2024-01-01                                 65.483158   
2025-01-01                                 66.722950   
2026-01-01                                 67.714783   
2027-01-01                                 68.508250   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  21.461811  ...   
2024-01-01                                  23.102637  ...   
2025-01-01                                  24.415298  ...   
2026-01-01                                  25.465427  ...   
2027-01-01                                  26.305530  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       74.833416   
2024-01-01                                       67.395317   
2025-01-01                                       61.444837   
2026-01-01                                       56.684453   
2027-01-01                                       52.876146   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               6.887529       1.394316e+07   
2024-01-01                               7.031778       1.412911e+07   
2025-01-01                               7.147176       1.427787e+07   
2026-01-01                               7.239495       1.439688e+07   
2027-01-01                               7.313350       1.449209e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.492958   
2024-01-01                            3.482540   
2025-01-01                            3.474205   
2026-01-01                            3.467538   
2027-01-01                            3.462203   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.416761   
2024-01-01                                         -0.557501   
2025-01-01                                         -0.670092

In [18]:
df_kenya_forecast = forecast_all_indicators(df_kenya)
df_kenya_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 10.918214   
2024-01-01                                 11.064833   
2025-01-01                                 11.182129   
2026-01-01                                 11.275965   
2027-01-01                                 11.351034   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           0.245069   
2024-01-01                                           0.191185   
2025-01-01                                           0.148078   
2026-01-01                                           0.113592   
2027-01-01                                           0.086004   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.161746e+11               5.033186   
2024-01-01       1.193660e+11               5.287946   
2025-01-01       1.219192e+11               5.491754   
2026-01-01       1.239617e+11               5.654801   
2027-01-01       1.255957e+11               5.785238   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   2147.348356                          3.058822   
2024-01-01                   2186.045878                          3.327176   
2025-01-01                   2217.003895                          3.541859   
2026-01-01                   2241.770308                          3.713606   
2027-01-01                   2261.583439                          3.851003   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            4.984341         30.695325   
2024-01-01                            5.131019         31.087936   
2025-01-01                            5.248361         31.402026   
2026-01-01                            5.342235         31.653298   
2027-01-01                            5.417335         31.854315   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 19.777111   
2024-01-01                                 20.023103   
2025-01-01                                 20.219897   
2026-01-01                                 20.377333   
2027-01-01                                 20.503281   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   3.684227  ...   
2024-01-01                                   3.234744  ...   
2025-01-01                                   2.875159  ...   
2026-01-01                                   2.587490  ...   
2027-01-01                                   2.357355  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      128.526415   
2024-01-01                                      131.940346   
2025-01-01                                      134.671490   
2026-01-01                                      136.856406   
2027-01-01                                      138.604339   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              21.383818       5.429076e+07   
2024-01-01                              22.371097       5.486537e+07   
2025-01-01                              23.160921       5.532506e+07   
2026-01-01                              23.792779       5.569281e+07   
2027-01-01                              24.298266       5.598701e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.670439   
2024-01-01                            3.655657   
2025-01-01                            3.643831   
2026-01-01                            3.634371   
2027-01-01                            3.626803   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.296438   
2024-01-01                                         -0.398309   
2025-01-01                                         -0.479806

In [19]:
df_madagascar_forecast = forecast_all_indicators(df_madagascar)
df_madagascar_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 25.306366   
2024-01-01                                 25.338112   
2025-01-01                                 25.363510   
2026-01-01                                 25.383827   
2027-01-01                                 25.400082   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           2.750299   
2024-01-01                                           2.666574   
2025-01-01                                           2.599594   
2026-01-01                                           2.546010   
2027-01-01                                           2.503143   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.511066e+10               2.456302   
2024-01-01       1.541420e+10               2.855660   
2025-01-01       1.565704e+10               3.175147   
2026-01-01       1.585130e+10               3.430737   
2027-01-01       1.600672e+10               3.635209   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    507.546666                          0.049199   
2024-01-01                    511.337621                          0.465163   
2025-01-01                    514.370384                          0.797934   
2026-01-01                    516.796595                          1.064151   
2027-01-01                    518.737564                          1.277125   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            7.136480         60.675032   
2024-01-01                            7.411021         61.487528   
2025-01-01                            7.630654         62.137524   
2026-01-01                            7.806361         62.657522   
2027-01-01                            7.946926         63.073519   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 35.368667   
2024-01-01                                 36.149416   
2025-01-01                                 36.774015   
2026-01-01                                 37.273694   
2027-01-01                                 37.673438   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                 -13.168775  ...   
2024-01-01                                 -15.935808  ...   
2025-01-01                                 -18.149434  ...   
2026-01-01                                 -19.920335  ...   
2027-01-01                                 -21.337055  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       68.599104   
2024-01-01                                       73.175093   
2025-01-01                                       76.835884   
2026-01-01                                       79.764517   
2027-01-01                                       82.107423   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               6.046881       2.978852e+07   
2024-01-01                               6.152737       3.018169e+07   
2025-01-01                               6.237422       3.049623e+07   
2026-01-01                               6.305170       3.074786e+07   
2027-01-01                               6.359369       3.094916e+07   

            Urban population growth (annual %)  \
2023-01-01                            4.079845   
2024-01-01                            4.038474   
2025-01-01                            4.005377   
2026-01-01                            3.978900   
2027-01-01                            3.957718   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.311395   
2024-01-01                                         -0.470093   
2025-01-01                                         -0.597051

In [20]:
df_mauritania_forecast = forecast_all_indicators(df_mauritania)
df_mauritania_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 47.373722   
2024-01-01                                 49.276902   
2025-01-01                                 50.799446   
2026-01-01                                 52.017481   
2027-01-01                                 52.991909   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                          12.988654   
2024-01-01                                          15.098367   
2025-01-01                                          16.786137   
2026-01-01                                          18.136353   
2027-01-01                                          19.216527   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       9.843187e+09               3.112947   
2024-01-01       1.023569e+10               3.492791   
2025-01-01       1.054969e+10               3.796667   
2026-01-01       1.080089e+10               4.039767   
2027-01-01       1.100185e+10               4.234248   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   2076.344159                          0.495281   
2024-01-01                   2136.351763                          0.864143   
2025-01-01                   2184.357846                          1.159233   
2026-01-01                   2222.762712                          1.395306   
2027-01-01                   2253.486606                          1.584163   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            5.244885         99.842335   
2024-01-01                            4.836204        102.957348   
2025-01-01                            4.509259        105.449359   
2026-01-01                            4.247703        107.442968   
2027-01-01                            4.038458        109.037855   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 52.468613   
2024-01-01                                 53.680446   
2025-01-01                                 54.649913   
2026-01-01                                 55.425487   
2027-01-01                                 56.045946   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  -4.193617  ...   
2024-01-01                                  -7.930397  ...   
2025-01-01                                 -10.919820  ...   
2026-01-01                                 -13.311359  ...   
2027-01-01                                 -15.224591  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      130.612345   
2024-01-01                                      132.553127   
2025-01-01                                      134.105752   
2026-01-01                                      135.347853   
2027-01-01                                      136.341533   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               4.718511       4.761850e+06   
2024-01-01                               4.544102       4.828098e+06   
2025-01-01                               4.404574       4.881097e+06   
2026-01-01                               4.292952       4.923495e+06   
2027-01-01                               4.203654       4.957414e+06   

            Urban population growth (annual %)  \
2023-01-01                            3.953494   
2024-01-01                            3.927219   
2025-01-01                            3.906199   
2026-01-01                            3.889383   
2027-01-01                            3.875930   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.437050   
2024-01-01                                         -0.592896   
2025-01-01                                         -0.717572

In [21]:
df_mauritius_forecast = forecast_all_indicators(df_mauritius)
df_mauritius_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 48.805053   
2024-01-01                                 50.448968   
2025-01-01                                 51.764100   
2026-01-01                                 52.816205   
2027-01-01                                 53.657890   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.902958   
2024-01-01                                           1.745140   
2025-01-01                                           1.618885   
2026-01-01                                           1.517881   
2027-01-01                                           1.437078   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.201581e+10               3.060928   
2024-01-01       1.180750e+10               4.505762   
2025-01-01       1.164085e+10               5.661630   
2026-01-01       1.150753e+10               6.586323   
2027-01-01       1.140087e+10               7.326079   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   9504.858008                          3.263474   
2024-01-01                   9344.837369                          4.764141   
2025-01-01                   9216.820857                          5.964674   
2026-01-01                   9114.407648                          6.925100   
2027-01-01                   9032.477081                          7.693442   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            6.957511        106.694190   
2024-01-01                            8.310880        110.408224   
2025-01-01                            9.393575        113.379451   
2026-01-01                           10.259731        115.756433   
2027-01-01                           10.952656        117.658018   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 57.889137   
2024-01-01                                 59.959256   
2025-01-01                                 61.615351   
2026-01-01                                 62.940228   
2027-01-01                                 64.000128   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   4.384293  ...   
2024-01-01                                   5.822206  ...   
2025-01-01                                   6.972536  ...   
2026-01-01                                   7.892799  ...   
2027-01-01                                   8.629011  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      158.026621   
2024-01-01                                      160.247341   
2025-01-01                                      162.023916   
2026-01-01                                      163.445176   
2027-01-01                                      164.582184   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              44.790516       1.264290e+06   
2024-01-01                              45.337001       1.263726e+06   
2025-01-01                              45.774189       1.263275e+06   
2026-01-01                              46.123939       1.262915e+06   
2027-01-01                              46.403739       1.262626e+06   

            Urban population growth (annual %)  \
2023-01-01                           -0.082190   
2024-01-01                           -0.102552   
2025-01-01                           -0.118843   
2026-01-01                           -0.131875   
2027-01-01                           -0.142301   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.258316   
2024-01-01                                         -0.336687   
2025-01-01                                         -0.399384

In [22]:
df_morocco_forecast = forecast_all_indicators(df_morocco)
df_morocco_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 39.984322   
2024-01-01                                 41.803228   
2025-01-01                                 43.258353   
2026-01-01                                 44.422452   
2027-01-01                                 45.353732   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.336881   
2024-01-01                                           1.295977   
2025-01-01                                           1.263255   
2026-01-01                                           1.237077   
2027-01-01                                           1.216134   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.362594e+11               1.964939   
2024-01-01       1.378102e+11               2.196958   
2025-01-01       1.390509e+11               2.382572   
2026-01-01       1.400435e+11               2.531064   
2027-01-01       1.408375e+11               2.649858   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   3575.666095                          0.939621   
2024-01-01                   3597.202679                          1.181501   
2025-01-01                   3614.431945                          1.375006   
2026-01-01                   3628.215359                          1.529809   
2027-01-01                   3639.242089                          1.653652   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            2.577947         90.167195   
2024-01-01                            2.995448         94.376981   
2025-01-01                            3.329449         97.744809   
2026-01-01                            3.596650        100.439072   
2027-01-01                            3.810410        102.594482   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 50.182873   
2024-01-01                                 52.573753   
2025-01-01                                 54.486457   
2026-01-01                                 56.016620   
2027-01-01                                 57.240750   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  -1.748967  ...   
2024-01-01                                  -2.398016  ...   
2025-01-01                                  -2.917255  ...   
2026-01-01                                  -3.332646  ...   
2027-01-01                                  -3.664958  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      112.742030   
2024-01-01                                      107.916725   
2025-01-01                                      104.056480   
2026-01-01                                      100.968285   
2027-01-01                                       98.497729   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              46.162401       3.756339e+07   
2024-01-01                              47.587190       3.777703e+07   
2025-01-01                              48.727021       3.794794e+07   
2026-01-01                              49.638886       3.808466e+07   
2027-01-01                              50.368378       3.819404e+07   

            Urban population growth (annual %)  \
2023-01-01                            1.822356   
2024-01-01                            1.804126   
2025-01-01                            1.789541   
2026-01-01                            1.777874   
2027-01-01                            1.768540   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.266865   
2024-01-01                                         -0.359981   
2025-01-01                                         -0.434474

In [23]:
df_namibia_forecast = forecast_all_indicators(df_namibia)
df_namibia_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 37.014642   
2024-01-01                                 37.818892   
2025-01-01                                 38.462291   
2026-01-01                                 38.977011   
2027-01-01                                 39.388786   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           6.582223   
2024-01-01                                           8.071251   
2025-01-01                                           9.262473   
2026-01-01                                          10.215451   
2027-01-01                                          10.977834   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.220570e+10               2.743262   
2024-01-01       1.224283e+10               4.077540   
2025-01-01       1.227254e+10               5.144962   
2026-01-01       1.229630e+10               5.998900   
2027-01-01       1.231531e+10               6.682051   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   4722.721946                          1.206228   
2024-01-01                   4692.799897                          2.565746   
2025-01-01                   4668.862258                          3.653361   
2026-01-01                   4649.712146                          4.523452   
2027-01-01                   4634.392057                          5.219526   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            3.744590         90.534383   
2024-01-01                            3.932470         93.489576   
2025-01-01                            4.082775         95.853730   
2026-01-01                            4.203019         97.745053   
2027-01-01                            4.299214         99.258111   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 53.519741   
2024-01-01                                 55.670684   
2025-01-01                                 57.391438   
2026-01-01                                 58.768042   
2027-01-01                                 59.869325   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  23.150198  ...   
2024-01-01                                  30.245500  ...   
2025-01-01                                  35.921743  ...   
2026-01-01                                  40.462736  ...   
2027-01-01                                  44.095531  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      113.989657   
2024-01-01                                      113.758271   
2025-01-01                                      113.573162   
2026-01-01                                      113.425075   
2027-01-01                                      113.306605   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              16.670550       2.580110e+06   
2024-01-01                              14.584937       2.602915e+06   
2025-01-01                              12.916446       2.621159e+06   
2026-01-01                              11.581653       2.635754e+06   
2027-01-01                              10.513819       2.647431e+06   

            Urban population growth (annual %)  \
2023-01-01                            3.263157   
2024-01-01                            3.173327   
2025-01-01                            3.101463   
2026-01-01                            3.043973   
2027-01-01                            2.997980   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.389894   
2024-01-01                                         -0.509070   
2025-01-01                                         -0.604411

In [24]:
df_rwanda_forecast = forecast_all_indicators(df_rwanda)
df_rwanda_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 21.726813   
2024-01-01                                 21.904901   
2025-01-01                                 22.047371   
2026-01-01                                 22.161346   
2027-01-01                                 22.252527   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           2.238267   
2024-01-01                                           2.245691   
2025-01-01                                           2.251630   
2026-01-01                                           2.256382   
2027-01-01                                           2.260183   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       1.262470e+10               7.502524   
2024-01-01       1.317577e+10               8.059573   
2025-01-01       1.361662e+10               8.505212   
2026-01-01       1.396930e+10               8.861723   
2027-01-01       1.425145e+10               9.146932   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    912.908791                          5.025354   
2024-01-01                    942.836658                          5.588617   
2025-01-01                    966.778952                          6.039227   
2026-01-01                    985.932787                          6.399715   
2027-01-01                   1001.255855                          6.688106   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                           11.093444         59.463921   
2024-01-01                           12.975637         60.108930   
2025-01-01                           14.481392         60.624936   
2026-01-01                           15.685996         61.037742   
2027-01-01                           16.649679         61.367986   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 37.737108   
2024-01-01                                 38.204029   
2025-01-01                                 38.577566   
2026-01-01                                 38.876395   
2027-01-01                                 39.115459   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   0.627542  ...   
2024-01-01                                  -1.423596  ...   
2025-01-01                                  -3.064506  ...   
2026-01-01                                  -4.377234  ...   
2027-01-01                                  -5.427417  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       80.828520   
2024-01-01                                       81.178689   
2025-01-01                                       81.458823   
2026-01-01                                       81.682931   
2027-01-01                                       81.862217   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               6.984768       1.385433e+07   
2024-01-01                               7.055035       1.403244e+07   
2025-01-01                               7.111248       1.417492e+07   
2026-01-01                               7.156219       1.428891e+07   
2027-01-01                               7.192195       1.438010e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.230445   
2024-01-01                            3.265010   
2025-01-01                            3.292662   
2026-01-01                            3.314783   
2027-01-01                            3.332480   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.312541   
2024-01-01                                         -0.431103   
2025-01-01                                         -0.525953

In [25]:
df_senegal_forecast = forecast_all_indicators(df_senegal)
df_senegal_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 25.737365   
2024-01-01                                 26.397189   
2025-01-01                                 26.925049   
2026-01-01                                 27.347337   
2027-01-01                                 27.685167   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                          10.297171   
2024-01-01                                          11.189187   
2025-01-01                                          11.902800   
2026-01-01                                          12.473690   
2027-01-01                                          12.930402   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       2.842395e+10               3.759061   
2024-01-01       2.927893e+10               3.562851   
2025-01-01       2.996292e+10               3.405884   
2026-01-01       3.051011e+10               3.280310   
2027-01-01       3.094786e+10               3.179851   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   1638.123300                          1.101269   
2024-01-01                   1667.153419                          0.932618   
2025-01-01                   1690.377514                          0.797696   
2026-01-01                   1708.956790                          0.689760   
2027-01-01                   1723.820211                          0.603410   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            6.114482         76.260323   
2024-01-01                            7.352965         79.479672   
2025-01-01                            8.343751         82.055151   
2026-01-01                            9.136379         84.115535   
2027-01-01                            9.770482         85.763841   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 50.522958   
2024-01-01                                 53.082483   
2025-01-01                                 55.130102   
2026-01-01                                 56.768198   
2027-01-01                                 58.078674   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  12.756532  ...   
2024-01-01                                  13.122025  ...   
2025-01-01                                  13.414420  ...   
2026-01-01                                  13.648336  ...   
2027-01-01                                  13.835468  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      121.057777   
2024-01-01                                      122.807972   
2025-01-01                                      124.208128   
2026-01-01                                      125.328252   
2027-01-01                                      126.224352   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              16.854170       1.742366e+07   
2024-01-01                              17.496434       1.767244e+07   
2025-01-01                              18.010246       1.787146e+07   
2026-01-01                              18.421296       1.803067e+07   
2027-01-01                              18.750135       1.815805e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.595232   
2024-01-01                            3.578049   
2025-01-01                            3.564303   
2026-01-01                            3.553306   
2027-01-01                            3.544509   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.327331   
2024-01-01                                         -0.451835   
2025-01-01                                         -0.551438

In [26]:
df_southafrica_forecast = forecast_all_indicators(df_southafrica)
df_southafrica_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 32.409904   
2024-01-01                                 33.465787   
2025-01-01                                 34.310494   
2026-01-01                                 34.986259   
2027-01-01                                 35.526872   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           5.235823   
2024-01-01                                           5.704439   
2025-01-01                                           6.079331   
2026-01-01                                           6.379246   
2027-01-01                                           6.619177   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       4.078074e+11               1.601931   
2024-01-01       4.146208e+11               2.137861   
2025-01-01       4.200715e+11               2.566605   
2026-01-01       4.244321e+11               2.909600   
2027-01-01       4.279205e+11               3.183996   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   6789.662661                          0.655749   
2024-01-01                   6869.402383                          1.222035   
2025-01-01                   6933.194161                          1.675064   
2026-01-01                   6984.227583                          2.037486   
2027-01-01                   7025.054321                          2.327425   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            5.363741         60.351062   
2024-01-01                            5.393002         62.225928   
2025-01-01                            5.416411         63.725821   
2026-01-01                            5.435138         64.925735   
2027-01-01                            5.450119         65.885666   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 27.941158   
2024-01-01                                 28.760140   
2025-01-01                                 29.415327   
2026-01-01                                 29.939475   
2027-01-01                                 30.358795   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   7.213071  ...   
2024-01-01                                  10.717182  ...   
2025-01-01                                  13.520470  ...   
2026-01-01                                  15.763101  ...   
2027-01-01                                  17.557205  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      169.329155   
2024-01-01                                      170.067469   
2025-01-01                                      170.658120   
2026-01-01                                      171.130641   
2027-01-01                                      171.508658   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              23.412342       6.010986e+07   
2024-01-01                              22.976044       6.044890e+07   
2025-01-01                              22.627006       6.072013e+07   
2026-01-01                              22.347775       6.093712e+07   
2027-01-01                              22.124390       6.111071e+07   

            Urban population growth (annual %)  \
2023-01-01                            1.653217   
2024-01-01                            1.610171   
2025-01-01                            1.575734   
2026-01-01                            1.548185   
2027-01-01                            1.526145   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.296535   
2024-01-01                                         -0.382773   
2025-01-01                                         -0.451763

In [27]:
df_sudan_forecast = forecast_all_indicators(df_sudan)
df_sudan_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                  1.945044   
2024-01-01                                  1.024174   
2025-01-01                                  0.287477   
2026-01-01                                 -0.301879   
2027-01-01                                 -0.773365   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.254575   
2024-01-01                                           0.974740   
2025-01-01                                           0.750872   
2026-01-01                                           0.571778   
2027-01-01                                           0.428502   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       4.145734e+10              -2.537918   
2024-01-01       4.433436e+10              -2.537484   
2025-01-01       4.663597e+10              -2.537137   
2026-01-01       4.847726e+10              -2.536859   
2027-01-01       4.995029e+10              -2.536637   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    861.714000                         -4.886364   
2024-01-01                    910.564977                         -4.685132   
2025-01-01                    949.645758                         -4.524146   
2026-01-01                    980.910383                         -4.395357   
2027-01-01                   1005.922083                         -4.292326   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                          179.713400          2.438551   
2024-01-01                          194.085872         -0.566432   
2025-01-01                          205.583849         -2.970418   
2026-01-01                          214.782230         -4.893607   
2027-01-01                          222.140936         -6.432158   

            Imports of goods and services (% of GDP)  \
2023-01-01                                  0.493507   
2024-01-01                                 -1.590605   
2025-01-01                                 -3.257895   
2026-01-01                                 -4.591727   
2027-01-01                                 -5.658793   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  -0.340877  ...   
2024-01-01                                  -0.070030  ...   
2025-01-01                                   0.146648  ...   
2026-01-01                                   0.319990  ...   
2027-01-01                                   0.458664  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       76.002444   
2024-01-01                                       76.134245   
2025-01-01                                       76.239685   
2026-01-01                                       76.324037   
2027-01-01                                       76.391519   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              13.058940       4.725448e+07   
2024-01-01                              13.097933       4.795488e+07   
2025-01-01                              13.129128       4.851520e+07   
2026-01-01                              13.154084       4.896346e+07   
2027-01-01                              13.174048       4.932206e+07   

            Urban population growth (annual %)  \
2023-01-01                            3.599622   
2024-01-01                            3.543051   
2025-01-01                            3.497794   
2026-01-01                            3.461588   
2027-01-01                            3.432624   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.275512   
2024-01-01                                         -0.325002   
2025-01-01                                         -0.364593

In [28]:
df_tanzania_forecast = forecast_all_indicators(df_tanzania)
df_tanzania_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 14.472698   
2024-01-01                                 14.562209   
2025-01-01                                 14.633817   
2026-01-01                                 14.691104   
2027-01-01                                 14.736933   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.582866   
2024-01-01                                           1.610372   
2025-01-01                                           1.632377   
2026-01-01                                           1.649980   
2027-01-01                                           1.664063   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       7.627635e+10               3.591068   
2024-01-01       7.907822e+10               3.457319   
2025-01-01       8.131972e+10               3.350320   
2026-01-01       8.311291e+10               3.264721   
2027-01-01       8.454747e+10               3.196241   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   1196.956272                          0.653107   
2024-01-01                   1224.521817                          0.565798   
2025-01-01                   1246.574253                          0.495951   
2026-01-01                   1264.216202                          0.440074   
2027-01-01                   1278.329761                          0.395372   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            2.975901         31.554018   
2024-01-01                            2.893837         32.233758   
2025-01-01                            2.828185         32.777549   
2026-01-01                            2.775664         33.212582   
2027-01-01                            2.733647         33.560608   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 17.081320   
2024-01-01                                 17.671549   
2025-01-01                                 18.143732   
2026-01-01                                 18.521478   
2027-01-01                                 18.823675   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   8.958294  ...   
2024-01-01                                   9.542479  ...   
2025-01-01                                  10.009827  ...   
2026-01-01                                  10.383705  ...   
2027-01-01                                  10.682807  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       90.873990   
2024-01-01                                       93.109162   
2025-01-01                                       94.897299   
2026-01-01                                       96.327809   
2027-01-01                                       97.472217   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               5.300469       6.597013e+07   
2024-01-01                               5.397845       6.702900e+07   
2025-01-01                               5.475746       6.787609e+07   
2026-01-01                               5.538067       6.855376e+07   
2027-01-01                               5.587924       6.909590e+07   

            Urban population growth (annual %)  \
2023-01-01                            4.885847   
2024-01-01                            4.820200   
2025-01-01                            4.767681   
2026-01-01                            4.725667   
2027-01-01                            4.692055   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.332017   
2024-01-01                                         -0.458703   
2025-01-01                                         -0.560051

In [29]:
df_togo_forecast = forecast_all_indicators(df_togo)
df_togo_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 24.307881   
2024-01-01                                 24.635910   
2025-01-01                                 24.898333   
2026-01-01                                 25.108271   
2027-01-01                                 25.276222   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                          -2.285945   
2024-01-01                                          -2.853582   
2025-01-01                                          -3.307691   
2026-01-01                                          -3.670979   
2027-01-01                                          -3.961609   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       8.432578e+09               5.368667   
2024-01-01       8.662239e+09               5.717931   
2025-01-01       8.845968e+09               5.997343   
2026-01-01       8.992951e+09               6.220872   
2027-01-01       9.110538e+09               6.399696   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    950.268654                          2.940117   
2024-01-01                    965.489341                          3.296549   
2025-01-01                    977.665890                          3.581695   
2026-01-01                    987.407130                          3.809811   
2027-01-01                    995.200122                          3.992305   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            3.703102         60.676077   
2024-01-01                            4.347829         62.224503   
2025-01-01                            4.863610         63.463244   
2026-01-01                            5.276235         64.454236   
2027-01-01                            5.606335         65.247030   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 36.368196   
2024-01-01                                 37.588593   
2025-01-01                                 38.564911   
2026-01-01                                 39.345965   
2027-01-01                                 39.970808   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  13.345503  ...   
2024-01-01                                  14.107129  ...   
2025-01-01                                  14.716429  ...   
2026-01-01                                  15.203870  ...   
2027-01-01                                  15.593823  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       73.940573   
2024-01-01                                       73.152559   
2025-01-01                                       72.522148   
2026-01-01                                       72.017819   
2027-01-01                                       71.614355   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              11.151046       8.897634e+06   
2024-01-01                              10.601672       9.011784e+06   
2025-01-01                              10.162173       9.103104e+06   
2026-01-01                               9.810574       9.176160e+06   
2027-01-01                               9.529295       9.234605e+06   

            Urban population growth (annual %)  \
2023-01-01                            3.621774   
2024-01-01                            3.604936   
2025-01-01                            3.591465   
2026-01-01                            3.580688   
2027-01-01                            3.572067   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.362853   
2024-01-01                                         -0.528522   
2025-01-01                                         -0.661058

In [30]:
df_tunisia_forecast = forecast_all_indicators(df_tunisia)
df_tunisia_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 47.362297   
2024-01-01                                 48.692541   
2025-01-01                                 49.756736   
2026-01-01                                 50.608092   
2027-01-01                                 51.289176   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           1.278070   
2024-01-01                                           1.183004   
2025-01-01                                           1.106952   
2026-01-01                                           1.046110   
2027-01-01                                           0.997437   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       4.505393e+10               1.350242   
2024-01-01       4.562750e+10               1.899757   
2025-01-01       4.608636e+10               2.339369   
2026-01-01       4.645344e+10               2.691058   
2027-01-01       4.674711e+10               2.972410   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   3633.170983                          0.591100   
2024-01-01                   3664.189667                          1.177074   
2025-01-01                   3689.004614                          1.645854   
2026-01-01                   3708.856572                          2.020877   
2027-01-01                   3724.738138                          2.320896   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            4.072348        106.357843   
2024-01-01                            3.466969        109.385925   
2025-01-01                            2.982666        111.808390   
2026-01-01                            2.595223        113.746363   
2027-01-01                            2.285269        115.296741   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 58.995545   
2024-01-01                                 60.693384   
2025-01-01                                 62.051655   
2026-01-01                                 63.138271   
2027-01-01                                 64.007565   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   1.904673  ...   
2024-01-01                                   4.036686  ...   
2025-01-01                                   5.742297  ...   
2026-01-01                                   7.106786  ...   
2027-01-01                                   8.198376  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      127.944117   
2024-01-01                                      129.009090   
2025-01-01                                      129.861069   
2026-01-01                                      130.542652   
2027-01-01                                      131.087918   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              36.534362       1.239711e+07   
2024-01-01                              37.588265       1.245510e+07   
2025-01-01                              38.431389       1.250149e+07   
2026-01-01                              39.105887       1.253860e+07   
2027-01-01                              39.645486       1.256829e+07   

            Urban population growth (annual %)  \
2023-01-01                            1.222483   
2024-01-01                            1.185437   
2025-01-01                            1.155800   
2026-01-01                            1.132090   
2027-01-01                            1.113123   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.275591   
2024-01-01                                         -0.369702   
2025-01-01                                         -0.444991

In [31]:
df_uganda_forecast = forecast_all_indicators(df_uganda)
df_uganda_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 13.876063   
2024-01-01                                 13.297624   
2025-01-01                                 12.834873   
2026-01-01                                 12.464672   
2027-01-01                                 12.168511   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           5.646492   
2024-01-01                                           6.198372   
2025-01-01                                           6.639876   
2026-01-01                                           6.993079   
2027-01-01                                           7.275641   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       4.472234e+10               3.982015   
2024-01-01       4.669944e+10               3.860105   
2025-01-01       4.828112e+10               3.762577   
2026-01-01       4.954646e+10               3.684554   
2027-01-01       5.055873e+10               3.622136   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    941.230758                          0.821565   
2024-01-01                    969.740054                          0.780250   
2025-01-01                    992.547491                          0.747198   
2026-01-01                   1010.793441                          0.720757   
2027-01-01                   1025.390200                          0.699603   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            3.950379         37.995476   
2024-01-01                            4.157472         37.826874   
2025-01-01                            4.323146         37.691992   
2026-01-01                            4.455685         37.584086   
2027-01-01                            4.561716         37.497762   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 24.119413   
2024-01-01                                 24.529250   
2025-01-01                                 24.857119   
2026-01-01                                 25.119414   
2027-01-01                                 25.329250   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   5.536929  ...   
2024-01-01                                   5.565150  ...   
2025-01-01                                   5.587726  ...   
2026-01-01                                   5.605787  ...   
2027-01-01                                   5.620236  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       68.561756   
2024-01-01                                       70.128943   
2025-01-01                                       71.382693   
2026-01-01                                       72.385693   
2027-01-01                                       73.188092   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               3.910609       4.765407e+07   
2024-01-01                               3.894749       4.847967e+07   
2025-01-01                               3.882061       4.914015e+07   
2026-01-01                               3.871911       4.966853e+07   
2027-01-01                               3.863791       5.009123e+07   

            Urban population growth (annual %)  \
2023-01-01                            5.422291   
2024-01-01                            5.328769   
2025-01-01                            5.253951   
2026-01-01                            5.194097   
2027-01-01                            5.146213   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.099142   
2024-01-01                                         -0.164960   
2025-01-01                                         -0.217614

In [32]:
df_congodemrep_forecast = forecast_all_indicators(df_congodemrep)
df_congodemrep_forecast

/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000gn/T/ipykernel_93895/2565535348.py:19: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  fit = model.fit(
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/pandas/util/_decorators.py:210: FutureWarning: the 'damping_slope' keyword is deprecated, use 'damping_trend' instead.
  return func(*args, **kwargs)
/Users/sahilbhardwaj/Desktop/UCL/Programming/Anaconda/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/var/folders/n7/10c67z5x7f34c4lv0zkn_5m40000

Exports of goods and services (% of GDP)  \
2023-01-01                                 42.834967   
2024-01-01                                 45.755090   
2025-01-01                                 48.091187   
2026-01-01                                 49.960066   
2027-01-01                                 51.455168   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           3.014317   
2024-01-01                                           3.106852   
2025-01-01                                           3.180881   
2026-01-01                                           3.240104   
2027-01-01                                           3.287482   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       6.340372e+10               7.178938   
2024-01-01       6.642374e+10               8.066726   
2025-01-01       6.883976e+10               8.776957   
2026-01-01       7.077258e+10               9.345142   
2027-01-01       7.231883e+10               9.799690   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                    638.668437                          3.842302   
2024-01-01                    659.917778                          4.716548   
2025-01-01                    676.917250                          5.415946   
2026-01-01                    690.516827                          5.975464   
2027-01-01                    701.396490                          6.423078   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            8.876536         86.462322   
2024-01-01                            7.288901         92.030961   
2025-01-01                            6.018794         96.485872   
2026-01-01                            5.002708        100.049800   
2027-01-01                            4.189839        102.900943   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 43.627355   
2024-01-01                                 46.275871   
2025-01-01                                 48.394684   
2026-01-01                                 50.089735   
2027-01-01                                 51.445775   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                  32.746246  ...   
2024-01-01                                  35.489219  ...   
2025-01-01                                  37.683598  ...   
2026-01-01                                  39.439101  ...   
2027-01-01                                  40.843503  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                       49.536988   
2024-01-01                                       50.948398   
2025-01-01                                       52.077526   
2026-01-01                                       52.980828   
2027-01-01                                       53.703470   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                               6.330318       9.967829e+07   
2024-01-01                               6.263341       1.013848e+08   
2025-01-01                               6.209759       1.027500e+08   
2026-01-01                               6.166894       1.038422e+08   
2027-01-01                               6.132602       1.047159e+08   

            Urban population growth (annual %)  \
2023-01-01                            4.455568   
2024-01-01                            4.438086   
2025-01-01                            4.424101   
2026-01-01                            4.412913   
2027-01-01                            4.403962   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.475927   
2024-01-01                                         -0.663151   
2025-01-01                                         -0.812931

In [34]:
import pandas as pd

forecast_dfs = [
    df_algeria_forecast, df_angola_forecast, df_benin_forecast, df_botswana_forecast,
    df_burkinafaso_forecast, df_burundi_forecast, df_cameroon_forecast, df_congodemrep_forecast,
    df_egyptarabrep_forecast, df_ghana_forecast, df_guinea_forecast, df_kenya_forecast,
    df_madagascar_forecast, df_mauritania_forecast, df_mauritius_forecast, df_morocco_forecast,
    df_namibia_forecast, df_rwanda_forecast, df_senegal_forecast, df_southafrica_forecast,
    df_sudan_forecast, df_tanzania_forecast, df_togo_forecast, df_tunisia_forecast, df_uganda_forecast
]

combined_forecast_df = pd.DataFrame()

for forecast_df in forecast_dfs:
    combined_forecast_df = pd.concat([combined_forecast_df, forecast_df])

combined_forecast_df.reset_index(drop=True, inplace=True)



combined_forecast_df.head(60)

Exports of goods and services (% of GDP)  \
0                                  27.502616   
1                                  29.572969   
2                                  31.229251   
3                                  32.554277   
4                                  33.614297   
5                                  47.649844   
6                                  49.462709   
7                                  50.913002   
8                                  52.073236   
9                                  53.001423   
10                                 19.987907   
11                                 19.023697   
12                                 18.252329   
13                                 17.635235   
14                                 17.141559   
15                                 38.578346   
16                                 39.229890   
17                                 39.751126   
18                                 40.168114   
19                                 40.501704   
20                                 33.001023   
21                                 33.579938   
22                                 34.043069   
23                                 34.413574   
24                                 34.709978   
25                                  4.637480   
26                                  4.617659   
27                                  4.601802   
28                                  4.589117   
29                                  4.578969   
30                                 17.436208   
31                                 17.599007   
32                                 17.729247   
33                                 17.833438   
34                                 17.916791   
35                                 42.834967   
36                                 45.755090   
37                                 48.091187   
38                                 49.960066   
39                                 51.455168   
40                                 13.311946   
41                                 13.139239   
42                                 13.001073   
43                                 12.890540   
44                                 12.802114   
45                                 32.866638   
46                                 32.799213   
47                                 32.745274   
48                                 32.702122   
49                                 32.667601   
50                                 45.749929   
51                                 46.374431   
52                                 46.874033   
53                                 47.273715   
54                                 47.593460   
55                                 10.918214   
56                                 11.064833   
57                                 11.182129   
58                                 11.275965   
59                                 11.351034   

    Foreign direct investment, net inflows (% of GDP)  GDP (current US$)  \
0                                            0.260709       2.042146e+11   
1                                            0.152651       2.109502e+11   
2                                            0.066204       2.163386e+11   
3                                           -0.002953       2.206494e+11   
4                                           -0.058279       2.240980e+11   
5                                           -6.843840       8.324356e+10   
6                                           -6.848700       8.982723e+10   
7                                           -6.852589       9.509416e+10   
8                                           -6.855699       9.930771e+10   
9                                           -6.858187       1.026785e+11   
10                                           1.601875       1.810611e+10   
11                                           1.664045       1.871281e+10   
12                                           1.713780       1.919817e+10   
13         

In [35]:
unique_countries = combined_forecast_df['Country Name'].unique()

print(unique_countries)


['Algeria' 'Angola' 'Benin' 'Botswana' 'Burkina Faso' 'Burundi' 'Cameroon'
 'Congo, Dem. Rep.' 'Egypt, Arab Rep.' 'Ghana' 'Guinea' 'Kenya'
 'Madagascar' 'Mauritania' 'Mauritius' 'Morocco' 'Namibia' 'Rwanda'
 'Senegal' 'South Africa' 'Sudan' 'Tanzania' 'Togo' 'Tunisia' 'Uganda']


In [36]:
from scipy.stats import rankdata

def calculate_percentile_ranks_with_metadata(df):
    percentile_df = df.copy()
    for column in df.columns:
        if column not in ['Country Name', 'Time']:
            percentile_df[column] = rankdata(df[column], method='max') / len(df[column]) *100
    return percentile_df

percentile_df_with_metadata_P5 = calculate_percentile_ranks_with_metadata(combined_forecast_df)

percentile_df_with_metadata_P5.head(45)

Exports of goods and services (% of GDP)  \
0                                       48.0   
1                                       49.6   
2                                       50.4   
3                                       52.0   
4                                       59.2   
5                                       84.0   
6                                       88.0   
7                                       92.8   
8                                       96.8   
9                                       99.2   
10                                      32.0   
11                                      31.2   
12                                      30.4   
13                                      27.2   
14                                      24.8   
15                                      67.2   
16                                      68.8   
17                                      70.4   
18                                      72.0   
19                                      72.8   
20                                      56.8   
21                                      58.4   
22                                      60.0   
23                                      61.6   
24                                      62.4   
25                                       8.0   
26                                       7.2   
27                                       6.4   
28                                       5.6   
29                                       4.8   
30                                      25.6   
31                                      26.4   
32                                      28.0   
33                                      28.8   
34                                      29.6   
35                                      74.4   
36                                      78.4   
37                                      84.8   
38                                      89.6   
39                                      94.4   
40                                      19.2   
41                                      17.6   
42                                      16.8   
43                                      16.0   
44                                      14.4   

    Foreign direct investment, net inflows (% of GDP)  GDP (current US$)  \
0                                                20.0               88.8   
1                                                17.6               89.6   
2                                                14.4               90.4   
3                                                13.6               91.2   
4                                                12.8               92.0   
5                                                 4.0               76.0   
6                                                 3.2               77.6   
7                                                 2.4               78.4   
8                                                 1.6               79.2   
9                                                 0.8               80.0   
10                                               48.8               28.8   
11                                               54.4               29.6   
12                                               56.0               30.4   
13                                               58.4               32.0   
14                                               59.2               33.6   
15                                               12.0               31.2   
16                                               11.2               35.2   
17                                               10.4               36.8   
18                                                9.6               38.4   
19                                                8.8               40.8   
20                                               45.6               34.4   
21                                               60.0               36.0   
22                                             

In [37]:
percentile_df_with_metadata_P5.columns

Index(['Exports of goods and services (% of GDP)',
       'Foreign direct investment, net inflows (% of GDP)',
       'GDP (current US$)', 'GDP growth (annual %)',
       'GDP per capita (current US$)', 'GDP per capita growth (annual %)',
       'Inflation, GDP deflator (annual %)', 'Trade (% of GDP)',
       'Imports of goods and services (% of GDP)',
       'Gross capital formation (annual % growth)',
       'Unemployment, total (% of total labor force) (modeled ILO estimate)',
       'Net ODA received (% of GNI)',
       'Agriculture, forestry, and fishing, value added (annual % growth)',
       'Industry (including construction), value added (annual % growth)',
       'Services, value added (annual % growth)',
       'Employment in agriculture (% of total employment) (modeled ILO estimate)',
       'Employment in industry (% of total employment) (modeled ILO estimate)',
       'Employment in services (% of total employment) (modeled ILO estimate)',
       'Government Effectiveness:

In [38]:
import pandas as pd

higher_is_better = [
    'Exports of goods and services (% of GDP)','Foreign direct investment, net inflows (% of GDP)',
    'GDP (current US$)', 'GDP growth (annual %)',
    'GDP per capita (current US$)', 'GDP per capita growth (annual %)''Trade (% of GDP)',
    'Gross capital formation (annual % growth)',  'Services, value added (annual % growth)',
    'Industry (including construction), value added (annual % growth)',
    'Employment in industry (% of total employment) (modeled ILO estimate)',
    'Employment in services (% of total employment) (modeled ILO estimate)',
    'Government Effectiveness: Estimate', 'Regulatory Quality: Estimate',
    'Control of Corruption: Estimate','Net ODA received (% of GNI)',
    'Individuals using the Internet (% of population)','Population-Normalized Internet Usage Growth Rate',
    'Access to electricity (% of population)','Population-Normalized Electricity Improvement Rate',
    'Mobile cellular subscriptions (per 100 people)',
    'School enrollment, tertiary (% gross)', 'Population, total',
    'Urban population growth (annual %)', 'Total population ages 15-39 (% of total population)',
    
]

lower_is_better = [
    'Government Debt to GDP', 'Inflation, GDP deflator (annual %)', 'Imports of goods and services (% of GDP)',
    'Agriculture, forestry, and fishing, value added (annual % growth)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)',
    'Employment in agriculture (% of total employment) (modeled ILO estimate)'
]

bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
high = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,0.9, 1.0]
low = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

def apply_scoring(df, column, bins, high, low, lower_is_better):
    if column in lower_is_better:
        df[column] = pd.cut(df[column], bins=bins, labels=low, include_lowest=True)
    else:
        df[column] = pd.cut(df[column], bins=bins, labels=high, include_lowest=True)
    return df

def process_percentile_dataframe(percentile_df, higher_is_better, lower_is_better):
    scores_df = percentile_df.copy()
    indicators = [col for col in percentile_df.columns if col not in ['Country Name', 'Time']]
    for indicator in indicators:
        scores_df = apply_scoring(scores_df, indicator, bins, high, low, lower_is_better)
    return scores_df


In [39]:
SDFP5 = process_percentile_dataframe(percentile_df_with_metadata_P5, higher_is_better, lower_is_better)

SDFP5.head(40)

Exports of goods and services (% of GDP)  \
0                                       0.5   
1                                       0.5   
2                                       0.6   
3                                       0.6   
4                                       0.6   
5                                       0.9   
6                                       0.9   
7                                       1.0   
8                                       1.0   
9                                       1.0   
10                                      0.4   
11                                      0.4   
12                                      0.4   
13                                      0.3   
14                                      0.3   
15                                      0.7   
16                                      0.7   
17                                      0.8   
18                                      0.8   
19                                      0.8   
20                                      0.6   
21                                      0.6   
22                                      0.6   
23                                      0.7   
24                                      0.7   
25                                      0.1   
26                                      0.1   
27                                      0.1   
28                                      0.1   
29                                      0.1   
30                                      0.3   
31                                      0.3   
32                                      0.3   
33                                      0.3   
34                                      0.3   
35                                      0.8   
36                                      0.8   
37                                      0.9   
38                                      0.9   
39                                      1.0   

   Foreign direct investment, net inflows (% of GDP) GDP (current US$)  \
0                                                0.2               0.9   
1                                                0.2               0.9   
2                                                0.2               1.0   
3                                                0.2               1.0   
4                                                0.2               1.0   
5                                                0.1               0.8   
6                                                0.1               0.8   
7                                                0.1               0.8   
8                                                0.1               0.8   
9                                                0.1               0.8   
10                                               0.5               0.3   
11                                               0.6               0.3   
12                                               0.6               0.4   
13                                               0.6               0.4   
14                                               0.6               0.4   
15                                               0.2               0.4   
16                                               0.2               0.4   
17                                               0.2               0.4   
18                                               0.1               0.4   
19                                               0.1               0.5   
20                                               0.5               0.4   
21                                               0.6               0.4   
22                                               0.7               0.4   
23                                               0.8               0.4   
24                                               0.8               0.5   
25                                               0.3               0.1   
26                                               0.3               0.1   
27 

In [41]:
desired_order = [
    'Country Name', 'Time',
    'Government Debt to GDP',
    'Exports of goods and services (% of GDP)',
    'Foreign direct investment, net inflows (% of GDP)',
    'GDP (current US$)',
    'GDP growth (annual %)',
    'GDP per capita (current US$)',
    'GDP per capita growth (annual %)',
    'Inflation, GDP deflator (annual %)',
    'Trade (% of GDP)',
    'Imports of goods and services (% of GDP)',
    'Gross capital formation (annual % growth)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)',
    'Net ODA received (% of GNI)',
    'Agriculture, forestry, and fishing, value added (annual % growth)',
    'Industry (including construction), value added (annual % growth)',
    'Services, value added (annual % growth)',
    'Employment in agriculture (% of total employment) (modeled ILO estimate)',
    'Employment in industry (% of total employment) (modeled ILO estimate)',
    'Employment in services (% of total employment) (modeled ILO estimate)',
    'Government Effectiveness: Estimate',
    'Regulatory Quality: Estimate',
    'Control of Corruption: Estimate',
    'Individuals using the Internet (% of population)',
    'Access to electricity (% of population)',
    'Population-Normalized Internet Usage Growth Rate',
    'Population-Normalized Electricity Improvement Rate',
    'Mobile cellular subscriptions (per 100 people)',
    'School enrollment, tertiary (% gross)',
    'Total population ages 15-39 (% of total population)',
    'Population, total',
    'Urban population growth (annual %)',
    
]

DFSP5 = SDFP5[desired_order]

DFSP5.head(20)

Country Name    Time Government Debt to GDP  \
0       Algeria  2023.0                    0.6   
1       Algeria  2024.0                    0.6   
2       Algeria  2025.0                    0.5   
3       Algeria  2026.0                    0.5   
4       Algeria  2027.0                    0.5   
5        Angola  2023.0                    0.4   
6        Angola  2024.0                    0.5   
7        Angola  2025.0                    0.6   
8        Angola  2026.0                    0.6   
9        Angola  2027.0                    0.7   
10        Benin  2023.0                    0.7   
11        Benin  2024.0                    0.7   
12        Benin  2025.0                    0.6   
13        Benin  2026.0                    0.6   
14        Benin  2027.0                    0.6   
15     Botswana  2023.0                    1.0   
16     Botswana  2024.0                    1.0   
17     Botswana  2025.0                    0.9   
18     Botswana  2026.0                    0.9   
19     Botswana  2027.0                    0.9   

   Exports of goods and services (% of GDP)  \
0                                       0.5   
1                                       0.5   
2                                       0.6   
3                                       0.6   
4                                       0.6   
5                                       0.9   
6                                       0.9   
7                                       1.0   
8                                       1.0   
9                                       1.0   
10                                      0.4   
11                                      0.4   
12                                      0.4   
13                                      0.3   
14                                      0.3   
15                                      0.7   
16                                      0.7   
17                                      0.8   
18                                      0.8   
19                                      0.8   

   Foreign direct investment, net inflows (% of GDP) GDP (current US$)  \
0                                                0.2               0.9   
1                                                0.2               0.9   
2                                                0.2               1.0   
3                                                0.2               1.0   
4                                                0.2               1.0   
5                                                0.1               0.8   
6                                                0.1               0.8   
7                                                0.1               0.8   
8                                                0.1               0.8   
9                                                0.1               0.8   
10                                               0.5               0.3   
11                                               0.6               0.3   
12                                               0.6               0.4   
13                                               0.6               0.4   
14                                               0.6               0.4   
15                                               0.2               0.4   
16                                               0.2               0.4   
17                                               0.2               0.4   
18                                               0.1               0.4   
19                                               0.1               0.5   

   GDP growth (annual %) GDP per capita (current US$)  \
0                    0.2                          0.8   
1                    0.3                          0.8   
2                    0.5                          0.9   
3                    0.6                          0.9   
4                    0.7                          0.9   
5                    0.1                          0.7   
6         

In [42]:
mp5 = DFSP5.isnull().sum()
mp5

Country Name                                                                0
Time                                                                        0
Government Debt to GDP                                                      0
Exports of goods and services (% of GDP)                                    0
Foreign direct investment, net inflows (% of GDP)                           0
GDP (current US$)                                                           0
GDP growth (annual %)                                                       0
GDP per capita (current US$)                                                0
GDP per capita growth (annual %)                                            0
Inflation, GDP deflator (annual %)                                          0
Trade (% of GDP)                                                            0
Imports of goods and services (% of GDP)                                    0
Gross capital formation (annual % growth)                       

In [43]:
m1 = percentile_df_with_metadata_P5.isnull().sum()

In [44]:
m1

Exports of goods and services (% of GDP)                                    0
Foreign direct investment, net inflows (% of GDP)                           0
GDP (current US$)                                                           0
GDP growth (annual %)                                                       0
GDP per capita (current US$)                                                0
GDP per capita growth (annual %)                                            0
Inflation, GDP deflator (annual %)                                          0
Trade (% of GDP)                                                            0
Imports of goods and services (% of GDP)                                    0
Gross capital formation (annual % growth)                                   0
Unemployment, total (% of total labor force) (modeled ILO estimate)         0
Net ODA received (% of GNI)                                                 0
Agriculture, forestry, and fishing, value added (annual % growth

In [45]:
m2 = combined_forecast_df.isnull().sum()

In [46]:
m2

Exports of goods and services (% of GDP)                                    0
Foreign direct investment, net inflows (% of GDP)                           0
GDP (current US$)                                                           0
GDP growth (annual %)                                                       0
GDP per capita (current US$)                                                0
GDP per capita growth (annual %)                                            0
Inflation, GDP deflator (annual %)                                          0
Trade (% of GDP)                                                            0
Imports of goods and services (% of GDP)                                    0
Gross capital formation (annual % growth)                                   0
Unemployment, total (% of total labor force) (modeled ILO estimate)         0
Net ODA received (% of GNI)                                                 0
Agriculture, forestry, and fishing, value added (annual % growth

In [47]:
m3 = SDFP5.isnull().sum()
m3

Exports of goods and services (% of GDP)                                    0
Foreign direct investment, net inflows (% of GDP)                           0
GDP (current US$)                                                           0
GDP growth (annual %)                                                       0
GDP per capita (current US$)                                                0
GDP per capita growth (annual %)                                            0
Inflation, GDP deflator (annual %)                                          0
Trade (% of GDP)                                                            0
Imports of goods and services (% of GDP)                                    0
Gross capital formation (annual % growth)                                   0
Unemployment, total (% of total labor force) (modeled ILO estimate)         0
Net ODA received (% of GNI)                                                 0
Agriculture, forestry, and fishing, value added (annual % growth

In [48]:
DFSP5.to_csv('DFSP5.csv')

In [50]:
df_southafrica_forecast.head(7)

Exports of goods and services (% of GDP)  \
2023-01-01                                 32.409904   
2024-01-01                                 33.465787   
2025-01-01                                 34.310494   
2026-01-01                                 34.986259   
2027-01-01                                 35.526872   

            Foreign direct investment, net inflows (% of GDP)  \
2023-01-01                                           5.235823   
2024-01-01                                           5.704439   
2025-01-01                                           6.079331   
2026-01-01                                           6.379246   
2027-01-01                                           6.619177   

            GDP (current US$)  GDP growth (annual %)  \
2023-01-01       4.078074e+11               1.601931   
2024-01-01       4.146208e+11               2.137861   
2025-01-01       4.200715e+11               2.566605   
2026-01-01       4.244321e+11               2.909600   
2027-01-01       4.279205e+11               3.183996   

            GDP per capita (current US$)  GDP per capita growth (annual %)  \
2023-01-01                   6789.662661                          0.655749   
2024-01-01                   6869.402383                          1.222035   
2025-01-01                   6933.194161                          1.675064   
2026-01-01                   6984.227583                          2.037486   
2027-01-01                   7025.054321                          2.327425   

            Inflation, GDP deflator (annual %)  Trade (% of GDP)  \
2023-01-01                            5.363741         60.351062   
2024-01-01                            5.393002         62.225928   
2025-01-01                            5.416411         63.725821   
2026-01-01                            5.435138         64.925735   
2027-01-01                            5.450119         65.885666   

            Imports of goods and services (% of GDP)  \
2023-01-01                                 27.941158   
2024-01-01                                 28.760140   
2025-01-01                                 29.415327   
2026-01-01                                 29.939475   
2027-01-01                                 30.358795   

            Gross capital formation (annual % growth)  ...  \
2023-01-01                                   7.213071  ...   
2024-01-01                                  10.717182  ...   
2025-01-01                                  13.520470  ...   
2026-01-01                                  15.763101  ...   
2027-01-01                                  17.557205  ...   

            Mobile cellular subscriptions (per 100 people)  \
2023-01-01                                      169.329155   
2024-01-01                                      170.067469   
2025-01-01                                      170.658120   
2026-01-01                                      171.130641   
2027-01-01                                      171.508658   

            School enrollment, tertiary (% gross)  Population, total  \
2023-01-01                              23.412342       6.010986e+07   
2024-01-01                              22.976044       6.044890e+07   
2025-01-01                              22.627006       6.072013e+07   
2026-01-01                              22.347775       6.093712e+07   
2027-01-01                              22.124390       6.111071e+07   

            Urban population growth (annual %)  \
2023-01-01                            1.653217   
2024-01-01                            1.610171   
2025-01-01                            1.575734   
2026-01-01                            1.548185   
2027-01-01                            1.526145   

            Population-Normalized Internet Usage Growth Rate  \
2023-01-01                                         -0.296535   
2024-01-01                                         -0.382773   
2025-01-01                                         -0.451763